In [ ]:
##################################################
# T2D MVMR PGRS Analysis  
#
#	T2D GWAS (trans-ethnic GWAS results, non-overlapping UKBB)
#
# Date:         May 24, 2021
#
# Adapted from:           rsalem
#
##################################################

library(stringr)
library(tidyverse)
library(R.utils)
library(data.table)

# Reading in SNP Reference list w/ betas and SNP allele details (EA: effect allele, NEA: non-effect allele, EAF=Effect allele frequency)

#structure: snid, alt, ref, beta
snpinfo=read.table("/data/nrnb03/users/agarduno/jupyter/IRKD_SNP/T2D_MVMR_20SEP21.txt", header = TRUE, na.strings=c("",".","NA"))


# renaming alleles to Effect (EA) & Non-Effect (NEA)
	names(snpinfo)=c("rsid", "EA", "NEA", "Beta")
	head(snpinfo)         



# CHECKING & DEALING WITH NEGATIVE Betas (OR < 1): SETTING All BETAS to Positive & Flipping alleles as needed
	# Creating copy of Beta/EA/NEA variables (for reference, saved as BETAX, EAX & NEAX)
		snpinfo$BETAX=snpinfo$Beta
		snpinfo$EAX=as.character(snpinfo$EA)
		snpinfo$NEAX=as.character(snpinfo$NEA)

	# Flipping if beta is negative: Betas and EA/NEA
snpinfo$Beta <- ifelse(snpinfo$BETAX < 0,snpinfo$BETAX*-1,snpinfo$BETAX)
snpinfo$NEA <- ifelse(snpinfo$BETAX < 0,snpinfo$EAX,snpinfo$NEAX)
snpinfo$EA <- ifelse(snpinfo$BETAX < 0,snpinfo$NEAX,snpinfo$EAX)


# Reading in ARIC EA dosage and sample files
        # Dosage
	dose_UKBB_Lipid2=fread("/data/nrnb03/users/agarduno/jupyter/IRKD_SNP/Alexis_T2D_ScottMVMR_SNPs_subset.dosage.gz", header=FALSE)
	dose_UKBB_Lipid=as.data.frame(dose_UKBB_Lipid2)
	rm(dose_UKBB_Lipid2)
	dim(dose_UKBB_Lipid)
	dose_UKBB_Lipid[1:10, 1:10]

                # NOTE: The 1st 6 columns are structured as follow: "CHR SNPID(1) SNPID(2) POSITION Allele1(ref) Allele2(alt)", followed by SNP dosages (scaled from 0 - 2)
                # as counts of allele2 (note: dosages range from 0-2 to account for genotype imputation uncertainty), for each subject in file

                #Dose_SNPINFO <- geno_UKBB_Lipid[-1,1:6] ## THIS IS WRONG, don't remove first row (lose of first SNP)
                Dose_SNPINFO <- dose_UKBB_Lipid[,1:6]
                #colnames(Dose_SNPINFO)=c("SNPID", "rsid", "POSITION", "ref", "EA") ## keep allele coding as ref/alt - to keep track of SNP allele use in dosage
                names(Dose_SNPINFO)=c("CHR", "SNPID", "rsid", "POSITION", "ref", "alt")

                DOSAGE  <- as.data.frame(t(dose_UKBB_Lipid[,-c(1:6)])) 
                colnames(DOSAGE)<- as.character(Dose_SNPINFO[,3])

        head(Dose_SNPINFO)
        head(DOSAGE)

        # Sample
	sample_UKBB_Lipid=read.table("/data/nrnb03/users/agarduno/jupyter/IRKD_SNP/Alexis_T2D_MVMR_SNPs_subset.sample.gz", header=TRUE)
                sample_UKBB_LipidX=sample_UKBB_Lipid[-1,1:2]
                names(sample_UKBB_LipidX)=c("FID", "IID")

# Generating Combined file Dosage w/ Sample ID File
geno_UKBB_Lipid=cbind(sample_UKBB_LipidX, DOSAGE)
	rm(DOSAGE)  # removing large files


# Combining Reference SNPList with SNP info from Dosage File
        SNPINFOX=merge(Dose_SNPINFO, snpinfo, by="rsid", all=FALSE)

	dim(SNPINFOX)
	#340   9

	dim(Dose_SNPINFO)
	#340   6

	dim(snpinfo)
	#338   4

	#checking SNPINFOX for duplicate SNP using rsIDS
	subset(as.data.frame(table(SNPINFOX$rsid)), Freq==2)
	#         Var1 Freq
	#59  rs1215468    2
	#212 rs4930011    2
	
	#checking Dose_SNPINFO for duplicate SNP using rsIDs
	subset(as.data.frame(table(Dose_SNPINFO$rsid)), Freq==2)
        #	 Var1 Freq
	#59  rs1215468    2
	#212 rs4930011    2

	#checking Dose_SNPINFO for duplicate SNP using SNPID
	subset(as.data.frame(table(Dose_SNPINFO$SNPID)), Freq==2)
	#	[1] Var1 Freq
	#<0 rows> (or 0-length row.names) ## no matches -> 'SNPID' is unique for 340 SNPS, while rsID has two duplicates


	# checking details of duplicate rsIDs in SNPINFOX File
	SNPINFOX[(SNPINFOX$rsid=="rs1215468"),]
	#        rsid CHR           SNPID POSITION ref alt EA NEA       Beta
	#59 rs1215468  13 13:80707429_A_C 80707429   A   C  A   G 0.03342375
	#60 rs1215468  13 13:80707429_A_G 80707429   A   G  A   G 0.03342375

	# checking details of duplicate rsIDs in SNPINFOX File
	SNPINFOX[(SNPINFOX$rsid=="rs4930011"),]
	#         rsid CHR          SNPID POSITION ref alt EA NEA       Beta
	#213 rs4930011  11 11:2856658_C_G  2856658   C   G  G   C 0.0128372
	#214 rs4930011  11 11:2856658_C_T  2856658   C   T  G   C 0.01283723

	# The issue is these 2 SNPs are tri-allelic (rs1215468 has A/C/G alleles), triallelic SNPs
	# are split into two biallelic SNPs and are both retrieved when we retrieve on chr/pos (or rsID)
	# The PGS script fails because of the trialllelic SNPs which are split into two biallelic variants 
	# (with same rsID), which the PRS calculator fxn fail (not designed for this)


# Checking Allele Coding - Flip or not
        #  SNP orientation check
        SNPINFOX$OK  =(SNPINFOX$EA == SNPINFOX$alt)
        SNPINFOX$FLIP=(SNPINFOX$EA == SNPINFOX$ref) # fixed the error of two refs

	# Check if alleles match/overlap (if bad, need to review/double check) - CORRECTED
	SNPINFOX$CHECK="BAD_Alleles"
        SNPINFOX$CHECK[SNPINFOX$EA == SNPINFOX$alt  & SNPINFOX$NEA == SNPINFOX$ref ]="OK"
        SNPINFOX$CHECK[SNPINFOX$NEA == SNPINFOX$alt & SNPINFOX$EA == SNPINFOX$ref ]="OK"


	table(SNPINFOX$OK)
	table(SNPINFOX$FLIP)
	table(SNPINFOX$CHECK) #one bad allele


	SNPINFOX[(SNPINFOX$rsid=="rs9411425"),]

##################################################################################################
# The GRS script does not run due to two SNPs with same rsid -> need to remove problematic allele
# Need to corret (remove the 2 problamatic snps noted above) in dose_UKBB_Lipid using SNPID
##################################################################################################

grep("9:134868417_G_A",  Dose_SNPINFO$SNPID)
#554


	# testing if positions retrieve correct SNPs
	dose_UKBB_Lipid[c(554), 1:10]
	
	# Removing problematic SNPs
    dose_UKBB_LipidX=dose_UKBB_Lipid[-c(554),]

	Dose_SNPINFOX <- dose_UKBB_LipidX[,1:6]
	names(Dose_SNPINFOX)=c("CHR", "SNPID", "rsid", "POSITION", "ref", "alt")

	DOSAGEX  <- as.data.frame(t(dose_UKBB_LipidX[,-c(1:6)])) 
	colnames(DOSAGEX)<- as.character(Dose_SNPINFOX[,3])

       	# Generating Combined file Dosage w/ Sample ID File
        geno_UKBB_LipidX=cbind(sample_UKBB_LipidX, DOSAGEX)


# Combining Reference SNPList with SNP info from Dosage File
        SNPINFOX=merge(Dose_SNPINFOX, snpinfo, by="rsid", all=FALSE)

	# Checking Allele Coding - Flip or not
        #  SNP orientation check
        SNPINFOX$OK  =(SNPINFOX$EA == SNPINFOX$alt)
        #SNPINFOX$FLIP=(SNPINFOX$EA == SNPINFOX$ref.y) #update from just ref (two refs)
        SNPINFOX$FLIP=(SNPINFOX$EA == SNPINFOX$ref) # fixed the error of two refs

	# Check if alleles match/overlap (if bad, need to review/double check) - CORRECTED
	# Check if alleles match/overlap (if bad, need to review/double check)
	SNPINFOX$CHECK="BAD_Alleles"
        SNPINFOX$CHECK[SNPINFOX$EA == SNPINFOX$alt  || SNPINFOX$NEA == SNPINFOX$ref ]="OK"
        SNPINFOX$CHECK[SNPINFOX$NEA == SNPINFOX$alt || SNPINFOX$EA == SNPINFOX$ref ]="OK"

	head(SNPINFOX)

	table(SNPINFOX$OK)
	table(SNPINFOX$FLIP)
	table(SNPINFOX$CHECK)



# SNP Orientation Checker Function
    # Note: Checking and Aligning SNPs (Strand Orientation check/correction)

        SNP_Align_GRS=function(XDOSE, XINFO)
                {
                
                # Copying Dosage file (avoids internal loops/file reference within script)
                DOSE_POS=XDOSE[,c("FID", "IID", as.character(XINFO$rsid))]
                DOSE_WT=XDOSE[,c("FID", "IID", as.character(XINFO$rsid))]

                # Generating list of SNPs to loop through
                XSNPLIST=names(DOSE_POS)[-c(1:2)]
                NN=length(XSNPLIST)

                # Sets alleles so that all alleles are postively associated with outcome based on snpinfo file (positive beta)
                for(i in 1:NN)
                        {
                        # Additional formatting to insure exact SNP name match (avoids partial match issues)
                        xsnp=XSNPLIST[i]
                        ZZ=grep(paste("^", xsnp, "$", sep=""), XINFO$rsid)
                        snp_info=XINFO[ZZ,]

                        #print(i)
                        #print(xsnp)
                        if(snp_info$OK=="TRUE")
                                {
                                DOSE_POS[,xsnp]=XDOSE[,xsnp]
                                DOSE_WT[,xsnp]=snp_info$Beta*XDOSE[,xsnp]
                                }
                        if(snp_info$FLIP=="TRUE")
                                {
                                DOSE_POS[,xsnp]=abs(XDOSE[,xsnp] -2)
                                DOSE_WT[,xsnp]=snp_info$Beta*(abs(XDOSE[,xsnp] -2))
                                }

                        }

                # Creating GRS Variables (set to 'NA')
                XDOSE_POS=DOSE_POS
                XDOSE_POS$GRS_RAW=NA
                XDOSE_POS$GRS_WT=NA

                # Creating GRS (count of positive associated allele (raw) and weighted (WT) versions)
                XDOSE_POS$GRS_RAW=rowSums(DOSE_POS[,-c(1:2)], na.rm = FALSE, dims = 1)
                XDOSE_POS$GRS_WT =rowSums(DOSE_WT[ ,-c(1:2)], na.rm = FALSE, dims = 1)

		# Checking that all SNPS used in PRS
		NSNPi=nrow(XINFO)
		NSNPd=ncol(XDOSE[,!(names(XDOSE) %in% c("FID", "IID"))])
		NSNPp=ncol(DOSE_POS[,!(names(DOSE_POS) %in% c("FID", "IID"))])

		print("####################################")
		print(paste("Num NSNP in SNPINFO File  : ", NSNPi, sep=""))
		print(paste("Num NSNP in Dosage File   : ", NSNPd, sep=""))
		print(paste("Num NSNP used in PRS Calc : ", NSNPp, sep=""))
		print("####################################")

                return(XDOSE_POS)
                }




# Running GRS Check and Calculator Script (note: GRS added to end of file)
        UKBB_IR_GRS=SNP_Align_GRS(XDOSE=geno_UKBB_LipidX, XINFO=SNPINFOX)

write.table(UKBB_IR_GRS, file="~/UKBB_T2D_MVMR_GRS_REV_20SEP21.txt", quote=FALSE, sep="\t", row.names=FALSE)





head(SNPINFOX)
dim(SNPINFOX)
dim(snpinfo)
dim(Dose_SNPINFO)


